In [103]:
import helpers
import numpy as np
import re 

#helpers.npNicePrint(arr)

data = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

with open("input_20.txt", 'r') as f:
    data = f.read().rstrip()

#print(colors, designs)


def safe_get(grid, index):
    try:
        return grid[tuple(index)]
    except:
        return ""
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]

#np.prod(sum(data))    
#data = [list(map(int, x.split(" "))) for x in data.split('\n') if not x == ""]

#data = [(int(row.split(",")[0]),int(row.split(",")[1])) for row in data.split("\n")]

#grid, moves = data.split("\n\n")
#moves = moves.replace("\n", "")

grid = {(x,y): str(val) for y, r in enumerate(data.split('\n')) for x, val in enumerate(r)}
#grid
#data = [int(x) for x in data.split(" ")]


# Output the parsed data
#print(data)


#grid = {(x,y): "." for x in range(max_coor+1) for y in range(max_coor+1)}

#helpers.visualize_dict_grid(grid) 

In [98]:
import heapq

def heuristic(a, b):
    """Calculate Manhattan distance as the heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, goal):
    """A* algorithm to find the shortest path in a grid."""
    if grid[start] == '#' or grid[goal] == '#':
        return None  # Start or goal is blocked

    # Priority queue for the open set
    open_set = []
    heapq.heappush(open_set, (0, start))

    # Came from map for reconstructing the path
    came_from = {}

    # Cost from start to current node
    g_score = {node: float('inf') for node in grid}
    g_score[start] = 0

    # Total cost of the current node
    f_score = {node: float('inf') for node in grid}
    f_score[start] = heuristic(start, goal)

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        # Check neighbors
        neighbors = [
            (current[0] + dx, current[1] + dy)
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
        ]
        for neighbor in neighbors:
            if neighbor not in grid or grid[neighbor] == '#':
                continue

            tentative_g_score = g_score[current] + 1  # Assume uniform cost grid

            if tentative_g_score < g_score[neighbor]:
                # This path to neighbor is better
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                if neighbor not in [item[1] for item in open_set]:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None  # No path found

In [99]:
start = [k for k,v in grid.items() if v == "S"][0]
goal = [k for k,v in grid.items() if v == "E"][0]
golden_path = astar(grid, start, goal)
golden_path_len = len(golden_path)



In [100]:
# Find the boundary coordinates
min_x = min(coord[0] for coord in grid.keys())
max_x = max(coord[0] for coord in grid.keys())
min_y = min(coord[1] for coord in grid.keys())
max_y = max(coord[1] for coord in grid.keys())

# Collect edge coordinates
edges = set([
    coord for coord in grid.keys()
    if coord[0] == min_x or coord[0] == max_x or coord[1] == min_y or coord[1] == max_y
])

def get_neighbours(node):
    return  [(node[0] + dx, node[1] + dy)
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]

cheats = set()

for node in golden_path[:-1]:
    for neighbor in get_neighbours(node):
        if neighbor in edges or grid[neighbor] == '.':
            continue
        if sum([grid[neighbor2] in {".", "E", "S"} for neighbor2 in get_neighbours(neighbor) if not neighbor2 in edges]) >= 2:
            for neighbor2 in get_neighbours(neighbor):
                if not (neighbor2 in edges) and not (neighbor2 == node) and not (grid[neighbor2] == "#") and not (neighbor2, neighbor, node) in cheats:
                        cheats.add((node, neighbor, neighbor2))

In [101]:
times = {}

for i, x in enumerate(reversed(golden_path)):
    times[x] = (golden_path_len-i-1, i) #(s moved thus far, s until goal)
times

cheats_valid = set()

for c1,c2,c3 in cheats:
    cheats_valid.add((c1,c3))
cheats_valid

cheat_lengths = []

for cheat in cheats_valid:
    #print(cheat)
    #print(golden_path_len - (times[cheat[0]][0] + times[cheat[2]][1]))
    time_saved = golden_path_len-1 - (times[cheat[0]][0] + times[cheat[1]][1] + 2)
    if time_saved > 0:
        cheat_lengths.append(time_saved)
    
cheat_counts = dict(sorted(Counter(cheat_lengths).items(), key=lambda x: x[0]))
ans = 0
for k,v in cheat_counts.items():
    if k >= 100:
        ans += v
ans

1393

In [92]:
There are 14 cheats that save 2 picoseconds.
There are 14 cheats that save 4 picoseconds.
There are 2 cheats that save 6 picoseconds.
There are 4 cheats that save 8 picoseconds.
There are 2 cheats that save 10 picoseconds.
There are 3 cheats that save 12 picoseconds.
There is one cheat that saves 20 picoseconds.
There is one cheat that saves 36 picoseconds.
There is one cheat that saves 38 picoseconds.
There is one cheat that saves 40 picoseconds.
There is one cheat that saves 64 picoseconds.

{((1, 2), (2, 1)),
 ((1, 2), (3, 2)),
 ((1, 3), (3, 3)),
 ((1, 10), (2, 9)),
 ((2, 1), (3, 2)),
 ((2, 9), (3, 8)),
 ((2, 13), (1, 12)),
 ((3, 1), (5, 1)),
 ((3, 2), (4, 3)),
 ((3, 2), (5, 2)),
 ((3, 8), (4, 7)),
 ((3, 11), (1, 11)),
 ((3, 11), (3, 9)),
 ((3, 12), (1, 12)),
 ((3, 12), (2, 13)),
 ((4, 3), (5, 2)),
 ((4, 11), (3, 12)),
 ((5, 2), (6, 1)),
 ((5, 2), (7, 2)),
 ((5, 3), (7, 3)),
 ((5, 12), (3, 12)),
 ((5, 12), (4, 11)),
 ((5, 13), (3, 13)),
 ((6, 1), (7, 2)),
 ((6, 13), (5, 12)),
 ((7, 1), (9, 1)),
 ((7, 2), (9, 2)),
 ((7, 3), (9, 3)),
 ((7, 4), (9, 4)),
 ((7, 5), (9, 5)),
 ((7, 6), (8, 7)),
 ((7, 6), (9, 6)),
 ((7, 7), (5, 7)),
 ((7, 7), (7, 9)),
 ((7, 11), (5, 11)),
 ((7, 12), (5, 12)),
 ((7, 12), (6, 13)),
 ((8, 7), (8, 9)),
 ((8, 7), (9, 6)),
 ((8, 9), (7, 10)),
 ((9, 2), (10, 1)),
 ((9, 3), (11, 3)),
 ((9, 4), (11, 4)),
 ((9, 5), (11, 5)),
 ((9, 7), (9, 9)),
 ((9, 7), (11, 7)),
 ((9, 10), (7, 10)),
 ((9, 10), (8, 9)),
 ((9, 11), (7, 11)),
 ((9, 12), (7, 12)),
 ((9, 13), 

# Old method, when I didn't realize it was only one path... slow

In [72]:
# Find the boundary coordinates
min_x = min(coord[0] for coord in grid.keys())
max_x = max(coord[0] for coord in grid.keys())
min_y = min(coord[1] for coord in grid.keys())
max_y = max(coord[1] for coord in grid.keys())

# Collect edge coordinates
edges = set([
    coord for coord in grid.keys()
    if coord[0] == min_x or coord[0] == max_x or coord[1] == min_y or coord[1] == max_y
])

def get_neighbours(node):
    return  [(node[0] + dx, node[1] + dy)
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]

cheats = set()

for node in golden_path[:-1]:
    for neighbor in get_neighbours(node):
        if neighbor in edges or grid[neighbor] == '.':
            continue
        if sum([grid[neighbor2] in {".", "E", "S"} for neighbor2 in get_neighbours(neighbor) if not neighbor2 in edges]) >= 2:
            for neighbor2 in get_neighbours(neighbor):
                if not (neighbor2 in edges) and not (neighbor2 == node) and not (grid[neighbor2] == "#") and not (neighbor2, neighbor, node) in cheats:
                        cheats.add((node, neighbor, neighbor2))

#print(edges)

In [73]:
def get_neighbours(node):
    return  [(node[0] + dx, node[1] + dy)
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]

cheats = set()

for node in golden_path[:-1]:
    for neighbor in get_neighbours(node):
        if neighbor in edges or grid[neighbor] == '.':
            continue
        if sum([grid[neighbor2] in {".", "E", "S"} for neighbor2 in get_neighbours(neighbor) if not neighbor2 in edges]) >= 2:
            for neighbor2 in get_neighbours(neighbor):
                if not (neighbor2 in edges) and not (neighbor2 == node) and not (grid[neighbor2] == "#") and not (neighbor2, neighbor, node) in cheats:
                        cheats.add((node, neighbor, neighbor2))
                
        

In [74]:
cheats_deletes = set([x[1] for x in cheats])

len(cheats)
#len(cheats_deletes)

77

In [65]:
from collections import Counter
from tqdm import tqdm

golden_path_length = len(golden_path)

cheat_lengths = []

for cheat in tqdm(cheats_deletes):
    n_grid = grid.copy()
    n_grid[cheat] = "."
    n_path_length = len(astar(n_grid, start, goal))
    if n_path_length < golden_path_length:
        cheat_lengths.append(golden_path_length-n_path_length)
        

cheat_counts = dict(sorted(Counter(cheat_lengths).items(), key=lambda x: x[0]))
ans = 0
for k,v in cheat_counts.items():
    if k >= 100:
        ans += v
ans

100%|██████████████████████████████████████████████████████████████████████████████| 7826/7826 [04:55<00:00, 26.46it/s]


1393

In [28]:
There are 14 cheats that save 2 picoseconds.
There are 14 cheats that save 4 picoseconds.
There are 2 cheats that save 6 picoseconds.
There are 4 cheats that save 8 picoseconds.
There are 2 cheats that save 10 picoseconds.
There are 3 cheats that save 12 picoseconds.
There is one cheat that saves 20 picoseconds.
There is one cheat that saves 36 picoseconds.
There is one cheat that saves 38 picoseconds.
There is one cheat that saves 40 picoseconds.
There is one cheat that saves 64 picoseconds.

85

In [54]:
n_grid = grid.copy()
for c in cheats_deletes:
    n_grid[c] = "O"
    
helpers.visualize_dict_grid(n_grid)     

# # # # # # # # # # # # # # #
# . . . O . . . O . . . . . #
# . O . O . O . O . O O O . #
# S O . . . O . O . O . . . #
# # # # # # # . O . O . O O #
# # # # # # # . O . O . . . #
# # # # # # # . O . # O O . #
# # # . . E O . . . O . . . #
# # O . O # # O O O # . O O #
# . . . # # # . . . O . . . #
# . O O # # # . O . # O O . #
# . O . . . O . O . O . . . #
# . O . O . O . O . O . O # #
# . . . O . . . O . . . # # #
# # # # # # # # # # # # # # #
